In [39]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_probability as tfp
from tensorflow.keras.optimizers import Adam

In [11]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


# Data Loading

In [12]:
# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = y_train % 2  # Convert labels to 0 (even) and 1 (odd) 
y_train_original = y_train.copy()  # {original labels}
y_test = y_test % 2
y_test_original = y_test.copy()

# Mark a subset of positive instances as unlabeled
positive_indices = np.where(y_train == 1)[0]
unlabeled_indices = np.random.choice(positive_indices, size=int(len(positive_indices) * 0.8), replace=False)
y_train[unlabeled_indices] = 0  # Marking unlabeled now y_train is PU dataset


X = x_train  # Your feature matrix

# Preparing 's', the binary indicator vector
# Start by marking all instances as 'unlabeled' (0)
s = np.zeros_like(y_train)
# Then, mark the originally positive instances as 'labeled' (1), excluding those you've marked as unlabeled
s[positive_indices] = 1
s[unlabeled_indices] = 0 # Ensure that the subset you've chosen to remain unlabeled stays that way

print("Training labels distribution:", np.bincount(y_train))
print("Training original labels distribution:", np.bincount(y_train_original))

Training labels distribution: [53898  6102]
Training original labels distribution: [29492 30508]


# Pre-training

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization

# # First Classifier Model
# model_1 = Sequential([
#     Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),  # Layer 1
#     Activation('relu'),
#     Conv2D(64, (3, 3)),  # Layer 2
#     Activation('relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(64, (3, 3), padding='same'),  # Layer 3
#     Activation('relu'),
#     Conv2D(128, (3, 3)),  # Layer 4
#     Activation('relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Flatten(),
#     Dense(128),  # Layer 5
#     Activation('relu'),
#     Dense(1),  # Output Layer
#     Activation('sigmoid')
# ])

model_1 = Sequential([
    Conv2D(96, (3, 3), padding='same', input_shape=(28, 28, 1)),  # First convolutional layer with 96 filters
    Activation('relu'),
    Conv2D(192, (3, 3), padding='same'),  # Second convolutional layer with 192 filters
    Activation('relu'),
    Conv2D(10, (1, 1), padding='same'),  # Third convolutional layer with 10 filters
    Activation('relu'),
    Flatten(),
    Dense(100),  # Dense layer with 100 units
    Activation('relu'),
    Dense(100),  # Additional Dense layer to make it 6 layers in total
    Activation('relu'),
    Dense(1),  # Output layer for binary classification
    Activation('sigmoid')
])


model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_1 = model_1.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


C:\Users\FH-Deg\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.8955 - loss: 0.3038 - val_accuracy: 0.8992 - val_loss: 0.2722
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8983 - loss: 0.2666 - val_accuracy: 0.8992 - val_loss: 0.2680
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8989 - loss: 0.2603 - val_accuracy: 0.8992 - val_loss: 0.2670
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 31s 21ms/step - accuracy: 0.8973 - loss: 0.2581 - val_accuracy: 0.8991 - val_loss: 0.2814
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 29s 20ms/step - accuracy: 0.8993 - loss: 0.2492 - val_accuracy: 0.8989 - val_loss: 0.2755


### Probablistic outcome of classifer to pre-train policy network
 

In [27]:
probabilities = model_1.predict(x_train)
threshold = min(probabilities[y_train == 1])

# Convert probabilities to binary labels based on the threshold
predicted_labels = (probabilities >= threshold).astype(int)
clf_predicted_labels = predicted_labels.flatten().astype(int)

print("Classifer predicted labels distribution:", np.bincount(clf_predicted_labels))

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step


In [32]:
# Policy-Network Model

class PolicyGradientNetwork:
    def __init__(self):
        self.model = self._create_model()
    
    def _create_model(self):
        model = Sequential([
            Conv2D(96, (3, 3), padding='same', input_shape=(28, 28, 1)),
            Activation('relu'),
            Conv2D(10, (3, 3), padding='same'),
            Activation('relu'),
            Flatten(),
            Dense(100),
            Activation('relu'),
            Dense(100),  # Additional Dense layer
            Activation('relu'),
            Dense(1),  # Softmax output for binary classification
            Activation('sigmoid') # Hard labeling
        ])
        return model
    
    def get_model(self):
        return self.model

policy_network = PolicyGradientNetwork()
pn_model = policy_network.get_model()

pn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model with one-hot encoded labels
history_2 = pn_model.fit(x_train, clf_predicted_labels, epochs=5, batch_size=32, validation_split=0.2)

C:\Users\FH-Deg\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - accuracy: 0.8984 - loss: 0.2242 - val_accuracy: 0.9458 - val_loss: 0.1208
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 29s 20ms/step - accuracy: 0.9491 - loss: 0.1127 - val_accuracy: 0.9531 - val_loss: 0.1095
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 29s 19ms/step - accuracy: 0.9619 - loss: 0.0864 - val_accuracy: 0.9595 - val_loss: 0.0980
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.9707 - loss: 0.0671 - val_accuracy: 0.9632 - val_loss: 0.0897
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.9798 - loss: 0.0513 - val_accuracy: 0.9608 - val_loss: 0.1004


## Interactive learning

In [41]:

class Agent:
    def __init__(self, model, alpha=0.003, gamma=0.99, threshold=0.5):

        self.gamma = gamma
        self.lr = alpha
        # self.n_actions = n_actions
        self.state_m = []
        self.action_m = []
        self.reward_m = []
        self.policy = model
        self.threshold = threshold
        # self.policy = PolicyGradientNetwork()
        # self.policy.compile(optimizer=Adam(learning_rate=self.lr))

    def choose_action(self, state):
        # state = tf.convert_to_tensor([obs], dtype=tf.float32)
        # probs = self.policy(state)
        # action_probs = tfp.distributions.Categorical(probs=probs)
        # action = action_probs.sample()
        state = tf.convert_to_tensor([state], dtype=tf.float32)
        print(f"State: {state}")
        # probability = self.policy(state)
        probability = self.policy.predict(state)
        action = 1 if probability.numpy()[0] > self.threshold else 0
        return action

        return action.numpy()[0]

    def store(self, obs, action, reward):
        self.state_m.append(obs)
        self.action_m.append(action)
        self.reward_m.append(reward)

    def learn(self):
        actions = tf.convert_to_tensor(self.action_m, dtype=tf.float32)
        rewards = np.array(self.reward_m)

        G = np.zeros_like(rewards)
        for t in range(len(rewards)):
            G_sum = 0
            discount = 1
            for k in range(t, len(rewards)):
                G_sum += rewards[k] * discount
                discount *= self.gamma
            G[t] = G_sum
        
        with tf.GradientTape() as tape:
            loss = 0
            for idx, (g, state) in enumerate(zip(G, self.state_m)):
                state = tf.convert_to_tensor([state], dtype=tf.float32)
                probs = self.policy(state)
                action_probs = tfp.distributions.Categorical(probs=probs)
                log_prob = action_probs.log_prob(actions[idx])
                loss += -g * tf.squeeze(log_prob)

        gradient = tape.gradient(loss, self.policy.trainable_variables)
        self.policy.optimizer.apply_gradients(zip(gradient, self.policy.trainable_variables))

        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []

In [ ]:
import gym
from gym import spaces
import numpy as np

class CustomEnv(gym.Env):
    def __init__(self, x_train, labels, mnist_data, model, threshold=0.5):
        super(CustomEnv, self).__init__()
        self.mnist_data = mnist_data
        self.model = model
        self.x_train=x_test
        self.labels=labels
        self.threshold = threshold
        
        self.action_space = spaces.Discrete(2)  # Actions: 0 or 1
        self.observation_space = spaces.Box(low=0, high=255, shape=(28, 28, 1), dtype=np.uint8)

    def step(self, action):
        img, label = self._get_next_image()
        
        predicted_prob = self.model.predict(img.reshape(1, 28, 28, 1))[0][0]
        predicted_label = 1 if predicted_prob >= self.threshold else 0
        
        reward = self._get_reward(label, predicted_label, predicted_prob)
        
        self.current_step += 1
        done = self.current_step >= len(self.mnist_data)
        
        return img, reward, done, {}

    def reset(self):
        self.current_step = 0
        return self._get_next_image()[0]

    def _get_next_image(self):
        img, label = self.mnist_data[self.current_step]
        return img, label

    def _get_reward(self, true_label, predicted_label, predicted_prob):
        if true_label == predicted_label:
            return predicted_prob
        else:
            return 1 - predicted_prob

    def render(self, mode='human', close=False):
        pass  # Add rendering logic if needed


In [ ]:
env = CustomEnv(X_train.to_numpy(), y_train.to_numpy(), balanced_df)
# Create an instance of CustomEnv
# env = CustomEnv(balanced_df)

# Reset the environment to its initial state
observation = env.reset()

In [43]:
import gym
# if __name__ == '__main__':
agent = Agent(alpha=0.0005, gamma=0.99, model=pn_model)

env = gym.make('LunarLander-v2')
score_history = []

num_episodes = 10 #2000

for i in range(num_episodes):
    done = False
    score = 0
    observation = env.reset()
    print(f"Obeservation: {observation}")
    while not done:
        action = agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        agent.store(observation, action, reward)
        observation = observation_
        score += reward
    score_history.append(score)

    agent.learn()
    avg_score = np.mean(score_history[-100:])
    print('episode: ', i,'score: %.1f' % score,
        'average score %.1f' % avg_score)

DependencyNotInstalled: box2D is not installed, run `pip install gym[box2d]`

In [34]:
# loop here
for epoch in range(10):
    
    probabilities = pn_model.predict(x_train)
    threshold = min(probabilities[y_train == 1])

    # Convert probabilities to binary labels based on the threshold
    threshold =0.5
    predicted_labels = (probabilities >= threshold).astype(int)
    policy_predicted_labels = predicted_labels.flatten().astype(int)

    print(f"Policy predicted labels distribution: {epoch}", np.bincount(policy_predicted_labels))

    history_1 = model_1.fit(x_train, policy_predicted_labels, epochs=5, batch_size=32, validation_split=0.2)
    probabilities = pn_model.predict(x_train)
    threshold = min(probabilities[y_train == 1])
    predicted_labels = (probabilities >= threshold).astype(int)
    clf_predicted_labels = predicted_labels.flatten().astype(int)
    print(f"Classifer predicted labels distribution: {epoch}", np.bincount(clf_predicted_labels))





    


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step
Policy predicted labels distribution: 0 [14364 45636]
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8533 - loss: 0.2845 - val_accuracy: 0.9477 - val_loss: 0.1237
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.9473 - loss: 0.1188 - val_accuracy: 0.9519 - val_loss: 0.1111
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.9585 - loss: 0.0931 - val_accuracy: 0.9603 - val_loss: 0.0905
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 29s 19ms/step - accuracy: 0.9690 - loss: 0.0730 - val_accuracy: 0.9605 - val_loss: 0.0916
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 29s 19ms/step - accuracy: 0.9732 - loss: 0.0636 - val_accuracy: 0.9614 - val_loss: 0.1017
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step
Classifer predicted labels distribution: 0 [10728 49272]
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step
Policy predicted labels distribution: 1 [14364 45636]
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 31s 21

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

class PUClassifierNN(object):
    def __init__(self, model_1=None, model_2=None, n_folds=2):
        self.model_1 = model_1  # The first 6-layer CNN classifier
        self.model_2 = model_2  # The second policy-network reinforcement neural network
        self.n_folds = n_folds
        self.c = None  # To store the constant c representing the proportion of positive instances
        self.history_1 = None
        self.history_2 = None

    def fit_model_1(self, X, s):
        # Fit the first model on labeled data only
        X_labeled = X[s == 1]  # Use only labeled data (s == 1)
        y_labeled = np.ones((X_labeled.shape[0], 1))  # Labels are all 1 for positive class
        
        self.model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        self.history_1 = self.model_1.fit(X_labeled, y_labeled, epochs=300, batch_size=32, validation_split=0.1)

    def estimate_c(self, X, s):
        # Estimate the constant c using the first model
        X_unlabeled = X[s == 0]  # Use unlabeled data
        preds = self.model_1.predict(X_unlabeled)
        self.c = np.mean(preds)

    def fit_model_2(self, X, s):
        # Prepare the dataset for the second model
        X_positive = X[s == 1]
        X_unlabeled = X[s == 0]
        y_positive = np.ones((X_positive.shape[0], 1))
        y_unlabeled = self.model_1.predict(X_unlabeled)  # Use model_1 to predict

        # Combine positive and unlabeled data, considering unlabeled as both positive and negative
        X_train = np.concatenate([X_positive, X_unlabeled], axis=0)
        y_train = np.concatenate([y_positive, y_unlabeled], axis=0)

        # Sample weights to adjust the influence of unlabeled data
        w_unlabeled = (1 - self.c) / self.c * (y_unlabeled / (1 - y_unlabeled))
        # sample_weight = np.concatenate([np.ones(y_positive.shape), w_unlabeled[:, 0]], axis=0)
        # Flatten the array of ones to match the dimensions of w_unlabeled
        sample_weight = np.concatenate([np.ones(y_positive.shape[0]), w_unlabeled[:, 0]], axis=0)


        # Fit the second model on the combined dataset
        y_train_one_hot = to_categorical(y_train)
        self.model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        # self.model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        self.history_2 =self.model_2.fit(X_train, y_train_one_hot, sample_weight=sample_weight, epochs=300, batch_size=32, validation_split=0.1)

    def fit(self, X, s):
        # Fit the first model and estimate c
        self.fit_model_1(X, s)
        self.estimate_c(X, s)

        # Fit the second model using the estimated c
        self.fit_model_2(X, s)


# Instantiate the PUClassifierNN with the defined models
pu_classifier_nn = PUClassifierNN(model_1=model_1, model_2=model_2, n_folds=5)

# Fit the PUClassifierNN with your data (X, s), where X is your feature matrix and s is your binary indicator vector
pu_classifier_nn.fit(X, s)


In [ ]:
for epoch in range(epochs=3):
    for i in range(len(X)):
        state = X[i:i+1]  # Current "state" is the image
        action_probabilities = model_2.predict(state)  # Model predicts action probabilities
        action = np.argmax(action_probabilities)  # Choose action with highest probability

        true_label = s[i]
        reward = 1 if action == true_label else -1  # Simplified reward

        # Here you'd typically update your model based on the action taken and the reward received
        # This might involve calculating gradients and applying them to update the model weights
        # However, the exact mechanism would depend on the RL algorithm being used (e.g., Q-learning, policy gradients)

    # Optionally evaluate model performance on a validation set after each epoch


In [ ]:
print(pu_classifier_nn.history_1)

In [ ]:
print(pu_classifier_nn.history_2)
def plot_history(history):
    """Plot training and validation loss and accuracy."""
    fig, axs = plt.subplots(2, 1, figsize=(10, 10))
    axs[0].plot(history.history['loss'], label='train')
    axs[0].plot(history.history['val_loss'], label='validation')
    axs[0].set_title('Loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()

    axs[1].plot(history.history['accuracy'], label='train')
    axs[1].plot(history.history['val_accuracy'], label='validation')
    axs[1].set_title('Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()

    plt.show()

# Assuming you have stored the history of training your models as `history_1` and `history_2`
plot_history(pu_classifier_nn.history_1)  # For the first classifier
plot_history(pu_classifier_nn.history_2)  # For the second classifier

In [ ]:
# Assuming model_1 is your first classifier (6-layer CNN)
# X_train is your training images, y_train is your modified labels with 1 for labeled positive, 0 for unlabeled and negative

# Filter only labeled data
labeled_mask = (y_train == 1)
X_labeled = x_train[labeled_mask]
y_labeled = y_train[labeled_mask]

model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_1 = model_1.fit(X_labeled, y_labeled, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
probabilities = model_1.predict(x_train)  # Estimate probabilities for all data


In [ ]:
# The combined dataset for the second classifier will include all instances
X_combined = x_train
y_combined = y_train  # Here, 1s are labeled positive, 0s are unlabeled and negative

# The sample weights will be 1 for labeled positive instances, and the predicted probabilities for unlabeled/negative instances
sample_weights = np.ones_like(y_train, dtype=float)  # Start with all ones
sample_weights[y_train == 0] = probabilities[y_train == 0].flatten()  # Adjust weights for unlabeled/negative

# Reshape if your model expects a different input shape
X_combined = X_combined.reshape((-1, 28, 28, 1))  # Reshaping for CNN input, if needed


In [ ]:

# Compile the model for a multi-class classification problem
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert labels to one-hot encoded format
y_combined_one_hot = to_categorical(y_combined)

# Fit the model with one-hot encoded labels
history_2 = model_2.fit(X_combined, y_combined_one_hot, sample_weight=sample_weights, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
  # Reshape test data if needed
# test_loss, test_acc = model_2.evaluate(X_test_reshaped, y_test)  # Use true labels here for evaluation
# print(f'Test accuracy: {test_acc}')

from tensorflow.keras.utils import to_categorical

X_test_reshaped = x_test.reshape((-1, 28, 28, 1))
# One-hot encode y_test
y_test_one_hot = to_categorical(y_test)

# Now use the one-hot encoded labels for evaluation
test_loss, test_acc = model_2.evaluate(X_test_reshaped, y_test_one_hot)
print(f'Test accuracy: {test_acc}')


# Visual

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_images(images, labels, predictions=None):
    """Plot images with their labels. Show predictions if provided."""
    n = images.shape[0]
    sqrt_val = int(np.ceil(np.sqrt(n)))
    fig, axes = plt.subplots(sqrt_val, sqrt_val, figsize=(10, 10))
    
    for i, ax in enumerate(axes.flat):
        if i >= n:
            break
        ax.imshow(images[i], cmap='binary')
        
        if predictions is not None:
            ax.set_title(f"True: {labels[i]}\nPred: {predictions[i]}")
        else:
            ax.set_title(f"Label: {labels[i]}")
        
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

# Predictions from the second classifier
predictions = model_2.predict(X_test_reshaped)
predicted_classes = (predictions > 0.5).astype(int).flatten()  # Assuming binary classification

# Selecting a few images to display
indices = np.random.choice(range(len(x_test)), 25, replace=False)  # Randomly select 25 images
selected_images = x_test[indices]
selected_labels = y_test[indices]
selected_predictions = predicted_classes[indices]

# Plotting
plot_images(selected_images, selected_labels, selected_predictions)


In [ ]:
def plot_history(history):
    """Plot training and validation loss and accuracy."""
    fig, axs = plt.subplots(2, 1, figsize=(10, 10))
    axs[0].plot(history.history['loss'], label='train')
    axs[0].plot(history.history['val_loss'], label='validation')
    axs[0].set_title('Loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()

    axs[1].plot(history.history['accuracy'], label='train')
    axs[1].plot(history.history['val_accuracy'], label='validation')
    axs[1].set_title('Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()

    plt.show()

# Assuming you have stored the history of training your models as `history_1` and `history_2`
plot_history(history_1)  # For the first classifier
plot_history(history_2)  # For the second classifier
